In [283]:
import numpy as np
import random
import os
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [284]:
import numpy as np
import gym
from gym import spaces
import random
import gym
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.distributions import Normal
import matplotlib.pyplot as plt
class MultiDimensionalStochasticProgrammingEnv(gym.Env):
    np.random.seed(0)
    def __init__(self, num_products=3):
        super(MultiDimensionalStochasticProgrammingEnv, self).__init__()
        
        self.num_products = num_products
        
        # 动作空间：每个产品的补货数量（0到10）
        self.action_space = spaces.Box(low=0, high=300, shape=(num_products,), dtype=np.float32)
        
        # 状态空间：每个产品的当前库存数量
        self.observation_space = spaces.Box(low=0.0, high=300.0, shape=(num_products,), dtype=np.float32)
        self.sequence_t = 10
        self.max_inventory = 300  # 每个产品的最大库存
        self.current_inventory = np.full(num_products, 10)  # 初始库存
        self.profit_per_unit = 10  # 每个产品的利润
        self.store_cost_per_unit = 5  # 每个产品的储存成本
        self.day = 0  # 天数计数器
        self.max_days = 100  # 设定最大天数
        self.weight_list = list(np.random.dirichlet(np.ones(int(self.max_days/self.sequence_t)), size=1).reshape(-1, 1).flatten())
        
    def reset(self):
        self.current_inventory = np.full(self.num_products, 10)  # 重置每个产品的库存
        self.total_reward = 0
        self.day = 0  # 每个 episode 重置天数计数器
        return self.current_inventory
    
    def step(self, action):
        self.current_inventory = np.minimum(self.max_inventory, self.current_inventory + action)

        self.day += 1  # 计数天数增加
        
        # 随机生成每个产品的需求
        demand = np.random.normal(loc=150, scale=1, size=self.num_products)
        
        # 实际满足的需求
        satisfied_demand = np.minimum(self.current_inventory, demand)
        self.current_inventory = self.current_inventory.astype(np.float64)  # 确保类型为 float64
        self.current_inventory -= satisfied_demand
        
        reward = (np.sum(satisfied_demand * self.profit_per_unit) - np.sum(self.current_inventory * self.store_cost_per_unit))*self.weight_list[int(self.day/self.sequence_t)]
        self.total_reward += reward
        
        # 当天数达到最大天数时，结束 episode
        done = self.day >= self.max_days-1
        
        return self.current_inventory, reward, done, {}
    
    def render(self, mode='human'):
        print(f"Current Inventory: {self.current_inventory}, Total Reward: {self.total_reward}, Day: {self.day}")


In [285]:
np.random.randn(1,10).reshape(-1,1)
random_numbers = list(np.random.dirichlet(np.ones(10), size=1).reshape(-1, 1).flatten())
random_numbers

[0.13910977875943475,
 0.06673757821703034,
 0.0744302408670181,
 0.2303801706851375,
 0.006533481190117616,
 0.008083001971357874,
 0.0018110815447159095,
 0.15849169955337453,
 0.13351381650944208,
 0.1809091507023712]

In [286]:
import gym
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import rl_utils


class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)


class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [287]:
import torch
import torch.nn.functional as F

class PolicyNetContinuous(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNetContinuous, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc_mu = torch.nn.Linear(hidden_dim, action_dim)
        self.fc_std = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # 强制 mu 为正：使用 softplus 或 ReLU 激活函数
        mu = F.softplus(self.fc_mu(x))  # softplus 是一个平滑的 ReLU 函数，输出正值
        std = F.softplus(self.fc_std(x))  # 确保标准差为正
        return mu, std



class PPOContinuous:
    ''' 处理连续动作的PPO算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                 lmbda, epochs, eps, gamma, device):
        self.actor = PolicyNetContinuous(state_dim, hidden_dim,
                                         action_dim).to(device)
        self.critic = ValueNet(state_dim, hidden_dim).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),
                                                lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),
                                                 lr=critic_lr)
        self.gamma = gamma
        self.lmbda = lmbda
        self.epochs = epochs
        self.eps = eps
        self.device = device

    def take_action(self, state):
        # Ensure that state is a tensor and move to the correct device
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        
        # Get the mean (mu) and standard deviation (sigma) from the actor network
        mu, sigma = self.actor(state)
        
        # Ensure the output is of shape (batch_size, action_dim) which should be (1, 10)
        action_dist = torch.distributions.Normal(mu, sigma)
        
        # Sample one action from the distribution
        action = action_dist.sample()  # This will be of shape (1, action_dim) = (1, 10)
        # Convert the action tensor to a 1D array (of size action_dim)
        action = action.squeeze(0)  # Remove the batch dimension, resulting in a 10-dimensional tensor
        
        # Convert the action to a list and return it
        return action.tolist()


    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions'],
                               dtype=torch.float).to(self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)
        td_target = rewards + self.gamma * self.critic(next_states) * (1 -
                                                                       dones)
        td_delta = td_target - self.critic(states)
        advantage = rl_utils.compute_advantage(self.gamma, self.lmbda,
                                               td_delta.cpu()).to(self.device)
        mu, std = self.actor(states)
        action_dists = torch.distributions.Normal(mu.detach(), std.detach())
        # 动作是正态分布

        old_log_probs = action_dists.log_prob(actions)

        for _ in range(self.epochs):
            mu, std = self.actor(states)
            std = torch.clamp(std, min=1e-6)
            action_dists = torch.distributions.Normal(mu, std)
            log_probs = action_dists.log_prob(actions)
            ratio = torch.exp(log_probs - old_log_probs)
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1 - self.eps, 1 + self.eps) * advantage
            actor_loss = torch.mean(-torch.min(surr1, surr2))
            critic_loss = torch.mean(
                F.mse_loss(self.critic(states), td_target.detach()))
            self.actor_optimizer.zero_grad()
            self.critic_optimizer.zero_grad()
            actor_loss.backward()
            critic_loss.backward()
            self.actor_optimizer.step()
            self.critic_optimizer.step()

In [288]:
class Backbone(nn.Module):
    def __init__(self, time_steps, state_dim = 10,action_dim = 10, hidden_dim = 64):
        super().__init__()
        self.time_steps = time_steps
        self.denoise_net = torch.nn.Sequential(
            torch.nn.Linear(state_dim + action_dim + 1, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, action_dim)
        )

    def forward(self, state, noisy_action, time_step):
        # 将时间步与 state 和 noisy_action 结合
        time_embedding = time_step.view(-1, 1).float() / self.time_steps
        input_data = torch.cat([state, noisy_action, time_embedding], dim=1)
        # 输出去噪后的动作
        return self.denoise_net(input_data)


In [289]:
class Model(nn.Module):
    def __init__(self, device, beta_1, beta_T, T, state_dim = 10,action_dim = 10):
        '''
        The epsilon predictor of diffusion process.

        beta_1    : beta_1 of diffusion process
        beta_T    : beta_T of diffusion process
        T         : Diffusion Steps
        input_dim : a dimension of data

        '''
        super().__init__()
        self.device = device
        self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, end=beta_T, steps=T), dim = 0).to(device = device)
        self.backbone = Backbone(T, state_dim, action_dim)
        
        self.to(device = self.device)

    def loss_fn(self, x,state, idx=None):
        '''
        This function performed when only training phase.

        x          : real data if idx==None else perturbation data
        idx        : if None (training phase), we perturbed random index. Else (inference phase), it is recommended that you specify.

        '''
        output, epsilon, alpha_bar = self.forward(x,state, idx=idx, get_target=True)
        loss = (output - epsilon).square().mean()
        return loss

        
    def forward(self, x,state, idx=None, get_target=False):
        '''
        x          : real data if idx==None else perturbation data
        idx        : if None (training phase), we perturbed random index. Else (inference phase), it is recommended that you specify.
        get_target : if True (training phase), target and sigma is returned with output (epsilon prediction)

        '''

        if idx == None:
            idx = torch.randint(0, len(self.alpha_bars), (x.size(0), )).to(device = self.device)
            used_alpha_bars = self.alpha_bars[idx][:, None]
            epsilon = torch.randn_like(x)
            x_tilde = torch.sqrt(used_alpha_bars) * x + torch.sqrt(1 - used_alpha_bars) * epsilon
            
        else:
            idx = torch.Tensor([idx for _ in range(x.size(0))]).to(device = self.device).long()
            x_tilde = x
            

        
        output = self.backbone(x_tilde,state, idx)
        
        return (output, epsilon, used_alpha_bars) if get_target else output
        

In [290]:
class DiffusionProcess():
    def __init__(self, beta_1, beta_T, T, diffusion_fn, device, datadim):
        '''
        beta_1        : beta_1 of diffusion process
        beta_T        : beta_T of diffusion process
        T             : step of diffusion process
        diffusion_fn  : trained diffusion network
        datadim         : data dimension
        '''

        self.betas = torch.linspace(start = beta_1, end=beta_T, steps=T)
        self.alphas = 1 - self.betas
        self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, end=beta_T, steps=T), dim = 0).to(device = device)
        self.alpha_prev_bars = torch.cat([torch.Tensor([1]).to(device=device), self.alpha_bars[:-1]])
        self.datadim = datadim
        
        self.diffusion_fn = diffusion_fn
        self.device = device

    
    def _one_diffusion_step(self, x):
        '''
        x   : perturbated data
        '''
        for idx in reversed(range(len(self.alpha_bars))):
            noise = torch.zeros_like(x) if idx == 0 else torch.randn_like(x)
            sqrt_tilde_beta = torch.sqrt((1 - self.alpha_prev_bars[idx]) / (1 - self.alpha_bars[idx]) * self.betas[idx])
            predict_epsilon = self.diffusion_fn(x, idx)
            mu_theta_xt = torch.sqrt(1 / self.alphas[idx]) * (x - self.betas[idx] / torch.sqrt(1 - self.alpha_bars[idx]) * predict_epsilon)
            x = mu_theta_xt + sqrt_tilde_beta * noise
            yield x
    
    @torch.no_grad()
    def sampling(self, sampling_number, only_final=False):
        '''
        sampling_number : a number of generation
        only_final      : If True, return is an only output of final schedule step 
        '''
        sample = torch.randn([sampling_number,self.datadim]).to(device = self.device).squeeze()
        sampling_list = []
        
        final = None
        for idx, sample in enumerate(self._one_diffusion_step(sample)):
            final = sample
            if not only_final:
                sampling_list.append(final)

        return final if only_final else torch.stack(sampling_list)

In [291]:
class DiffusionProcess():
    def __init__(self, beta_1, beta_T, T, diffusion_fn, device, datadim):
        '''
        beta_1        : beta_1 of diffusion process
        beta_T        : beta_T of diffusion process
        T             : step of diffusion process
        diffusion_fn  : trained diffusion network
        datadim         : data dimension
        '''

        self.betas = torch.linspace(start = beta_1, end=beta_T, steps=T)
        self.alphas = 1 - self.betas
        self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, end=beta_T, steps=T), dim = 0).to(device = device)
        self.alpha_prev_bars = torch.cat([torch.Tensor([1]).to(device=device), self.alpha_bars[:-1]])
        self.datadim = datadim
        
        self.diffusion_fn = diffusion_fn
        self.device = device

    
    def _one_diffusion_step(self, x,state, idx=None):
        '''
        x   : perturbated data
        '''
        for idx in reversed(range(len(self.alpha_bars))):
            noise = torch.zeros_like(x) if idx == 0 else torch.randn_like(x)
            sqrt_tilde_beta = torch.sqrt((1 - self.alpha_prev_bars[idx]) / (1 - self.alpha_bars[idx]) * self.betas[idx])
            predict_epsilon = self.diffusion_fn(x, state,idx)
            mu_theta_xt = torch.sqrt(1 / self.alphas[idx]) * (x - self.betas[idx] / torch.sqrt(1 - self.alpha_bars[idx]) * predict_epsilon)
            x = mu_theta_xt + sqrt_tilde_beta * noise
            yield x
    
    @torch.no_grad()
    def sampling(self, state, sampling_number, only_final=False):
        '''
        sampling_number : a number of generation
        only_final      : If True, return is an only output of final schedule step 
        '''
        sample = torch.randn([sampling_number,self.datadim]).to(device = self.device).squeeze()
        sampling_list = []
        
        final = None
        for idx, sample in enumerate(self._one_diffusion_step(sample,state)):
            final = sample
            if not only_final:
                sampling_list.append(final)

        return final if only_final else torch.stack(sampling_list)

In [292]:
def train_ddpm(model, optimizer, num_epochs, state_loader, action_loader,state_loader_v, action_loader_v, early_stopping):
    best_loss = float('inf')
    early_stopping_counter = 0
    for epoch in range(num_epochs):
        whole_loss = 0
        for i, (state_batch, action_batch) in enumerate(zip(state_loader, action_loader)):
            state_batch = state_batch.cuda()
            action_batch = action_batch.cuda()
            loss = model.loss_fn(action_batch, state_batch)
            whole_loss += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        val_loss = 0
        with torch.no_grad():
            for state_val_batch, action_val_batch in zip(state_loader_v, action_loader_v):
                state_val_batch = state_val_batch.cuda()
                action_val_batch = action_val_batch.cuda()
                val_loss += model.loss_fn(action_val_batch, state_val_batch)
            val_loss /= len(state_loader)
        if (epoch) % 20 == 0:
            print('epoch: {}, Train Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch, whole_loss/len(state_loader), val_loss.item()))
        loss_new = val_loss
        if loss_new < best_loss:
            best_loss = loss_new
            early_stopping_counter = 0
            print('epoch: {}, find new best loss: Train Loss: {:.4f}'.format(epoch, best_loss))
            print('-' * 10)
        else:
            early_stopping_counter += 1
        if early_stopping_counter == early_stopping:
            print("Early stopping after {} epochs".format(epoch))
            break

In [293]:
import torch
import torch.nn.functional as F
import numpy as np


class PolicyNetContinuous(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNetContinuous, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc_mu = torch.nn.Linear(hidden_dim, action_dim)
        self.fc_std = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        mu = self.fc_mu(x)
        std = F.softplus(self.fc_std(x)) + 1e-3  # 确保标准差正值
        return mu, std




In [294]:
from tqdm import tqdm
import numpy as np
import torch
import collections
import random
env = MultiDimensionalStochasticProgrammingEnv(num_products=10)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high
actor_lr = 1e-4
critic_lr = 5e-3
num_episodes = 1000
hidden_dim = 128
gamma = 0.9
lmbda = 0.9
epochs = 10
eps = 0.2
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
agent = PPOContinuous(state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                      lmbda, epochs, eps, gamma, device)

In [295]:
from tqdm import tqdm
def train_on_policy_agent(env, agent, num_episodes):
    return_list = []
    transition_dict_whole = {'states': [], 'actions': [], 'next_states': [], 'rewards': [], 'dones': []}
    for i in range(5):
        with tqdm(total=int(num_episodes/10), desc='Iteration %d' % i) as pbar:
            for i_episode in range(int(num_episodes/10)):
                episode_return = 0
                transition_dict = {'states': [], 'actions': [], 'next_states': [], 'rewards': [], 'dones': []}
                state = env.reset()
                done = False
                while not done:
                    action = agent.take_action(state)
                    next_state, reward, done, _ = env.step(action)
                    transition_dict['states'].append(state)
                    transition_dict['actions'].append(action)
                    transition_dict['next_states'].append(next_state)
                    transition_dict['rewards'].append(reward)
                    transition_dict['dones'].append(done)
                    transition_dict_whole['states'].append(state)
                    transition_dict_whole['actions'].append(action)
                    transition_dict_whole['next_states'].append(next_state)
                    transition_dict_whole['rewards'].append(reward)
                    transition_dict_whole['dones'].append(done)
                    state = next_state
                    episode_return += reward
                return_list.append(episode_return)
                agent.update(transition_dict)
                if (i_episode+1) % 10 == 0:
                    pbar.set_postfix({'episode': '%d' % (num_episodes/10 * i + i_episode+1), 'return': '%.3f' % np.mean(return_list[-10:])})
                pbar.update(1)
    return return_list,transition_dict_whole

In [296]:
return_list,transition_dict = train_on_policy_agent(env, agent, num_episodes)

Iteration 4: 100%|██████████| 100/100 [00:08<00:00, 12.39it/s, episode=500, return=2344.127]


In [297]:
transition_dict['states'] = np.array(transition_dict['states'])
transition_dict['actions'] = np.array(transition_dict['actions'])

In [298]:
# 测试组合模型
state_dim = 10
hidden_dim = 64
action_dim = 10
time_steps = 1000
batch_size = 32

transition_dict['actions'] = np.array(transition_dict['actions'])
transition_dict['actions'].shape
state = torch.tensor(transition_dict['states'], dtype=torch.float).to(device)
action = torch.tensor(transition_dict['actions'], dtype=torch.float).to(device)
batch_size =64
beta_1 = 1e-4
beta_T = 0.02
T = 50
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
model = Model(device, beta_1, beta_T, T)
optim = torch.optim.Adam(model.parameters(), lr = 0.001)
model(state,action, idx=0, get_target=False).shape

torch.Size([49500, 10])

In [299]:
from sklearn.model_selection import train_test_split

# Convert tensors to numpy arrays
state_np = state.detach().cpu().numpy()
action_np = action.detach().cpu().numpy()
sc = StandardScaler()
action_np = sc.fit_transform(action_np)

# Define the sizes for training, validation, and test sets
train_size = 5000
val_size = 5000
test_size = 1000

# Split the data into training, validation, and test sets
state_train, state_temp, action_train, action_temp = train_test_split(state_np, action_np, train_size=train_size, random_state=42)
state_val, state_test, action_val, action_test = train_test_split(state_temp, action_temp, test_size=test_size, random_state=42)

# Convert the numpy arrays to DataLoader
state_train = DataLoader(state_train, batch_size=batch_size, shuffle=True)
state_val = DataLoader(state_val, batch_size=batch_size, shuffle=True)
state_test = DataLoader(state_test, batch_size=batch_size, shuffle=True)

action_train = DataLoader(action_train, batch_size=batch_size, shuffle=True)
action_val = DataLoader(action_val, batch_size=batch_size, shuffle=True)
action_test = DataLoader(action_test, batch_size=batch_size, shuffle=True)

In [300]:
train_ddpm(model, optim, 500, state_train, action_train,state_val, action_val, 100)

epoch: 0, Train Loss: 0.9839, Val Loss: 7.9200
epoch: 0, find new best loss: Train Loss: 7.9200
----------
epoch: 1, find new best loss: Train Loss: 7.4123
----------
epoch: 2, find new best loss: Train Loss: 7.0940
----------
epoch: 3, find new best loss: Train Loss: 6.9726
----------
epoch: 4, find new best loss: Train Loss: 6.9240
----------
epoch: 5, find new best loss: Train Loss: 6.8979
----------
epoch: 6, find new best loss: Train Loss: 6.8527
----------
epoch: 7, find new best loss: Train Loss: 6.8258
----------
epoch: 8, find new best loss: Train Loss: 6.8074
----------
epoch: 10, find new best loss: Train Loss: 6.8026
----------
epoch: 11, find new best loss: Train Loss: 6.8008
----------
epoch: 12, find new best loss: Train Loss: 6.7706
----------
epoch: 13, find new best loss: Train Loss: 6.7662
----------
epoch: 14, find new best loss: Train Loss: 6.7358
----------
epoch: 19, find new best loss: Train Loss: 6.7016
----------
epoch: 20, Train Loss: 0.7833, Val Loss: 6.7139

In [301]:
state_train, state_temp, action_train, action_temp = train_test_split(state_np, action_np, train_size=train_size, random_state=42)
state_val, state_test, action_val, action_test = train_test_split(state_temp, action_temp, test_size=test_size, random_state=42)
sampling_number = state_test.shape[0]
only_final = True
process = DiffusionProcess(beta_1, beta_T, T, model, device, action_dim)
action = process.sampling(torch.tensor(state_test).to(device=device),sampling_number, only_final)
action = sc.inverse_transform(action.cpu().numpy())
torch.save(model.state_dict(), 'ddpm_model.pth')

In [308]:
print('Mean Squared Error:', np.mean((action_test - action)**2))
print(action[4])

Mean Squared Error: 3.0509133
[ 0.1845298   1.4341173   1.299663   -0.07048453  0.70084965 -0.24861439
  1.3923907   0.9426981   1.1415417   0.22510754]
